# 2. Evaluación de Sistemas RAG con LangSmith

## Objetivos de Aprendizaje
- Comprender la importancia de la evaluación en sistemas RAG.
- Configurar LangSmith para trazabilidad y evaluación.
- Crear un dataset de evaluación con preguntas y respuestas de referencia.
- Ejecutar evaluadores automáticos para métricas como relevancia y fidelidad.
- Analizar los resultados de la evaluación para optimizar el sistema.

## ¿Qué es LangSmith?

LangSmith es una plataforma de LangChain para la observabilidad, el monitoreo y la evaluación de aplicaciones construidas con Modelos de Lenguaje Grandes (LLMs). Permite visualizar cada paso de una cadena o agente, analizar su rendimiento y evaluar la calidad de las respuestas de forma sistemática.

Para un sistema RAG, LangSmith nos ayuda a responder preguntas clave:
- **Recuperación (Retrieval)**: ¿Los documentos que encontramos son relevantes para la pregunta?
- **Generación (Generation)**: ¿La respuesta generada es fiel a los documentos recuperados? ¿Responde correctamente a la pregunta del usuario?

## 1. Instalación y Configuración

In [ ]:
!pip install --upgrade openai langsmith langchain langchain-openai langchain-community httpx scikit-learn numpy -q

In [ ]:
import os
from openai import OpenAI
from langsmith import Client
import json
from typing import Dict, Any, List
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

print("✅ Librerías importadas")

### Configuración de Variables de Entorno

Para que LangSmith capture las trazas de nuestra aplicación, necesitamos configurar cuatro variables de entorno:
1. `LANGCHAIN_TRACING_V2`: Se establece en `"true"` para activar la trazabilidad.
2. `LANGCHAIN_API_KEY`: Tu clave de API de LangSmith. La puedes obtener en [smith.langchain.com](https://smith.langchain.com).
3. `LANGCHAIN_PROJECT`: El nombre del proyecto bajo el cual se agruparán las trazas. Esto es muy útil para organizar el trabajo.
4. `OPENAI_API_KEY`: Tu clave de API de OpenAI para que el modelo de lenguaje funcione.

In [ ]:
# Configurar variables de entorno para LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "RAG-Evaluation-Tutorial"

# Si usas LangSmith, asegúrate de tener tu API key
# os.environ["LANGCHAIN_API_KEY"] = "tu_langsmith_api_key"

# Configurar API keys para el modelo de lenguaje
# Si usas OpenAI directamente:
# os.environ["OPENAI_API_KEY"] = "tu_openai_api_key"

# Si usas GitHub Models o Azure AI:
os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN", "")
os.environ["OPENAI_API_BASE"] = os.getenv("GITHUB_BASE_URL", "https://models.inference.ai.azure.com")

# Inicializar el cliente de LangSmith
try:
    client = Client()
    print("✅ Cliente de LangSmith configurado")
except Exception as e:
    print(f"⚠️ LangSmith no configurado: {e}")
    print("Continuando sin LangSmith...")
    client = None

print("✅ Variables de entorno configuradas")

## 2. Sistema RAG Básico

Reutilizaremos el sistema RAG simple del notebook anterior. Este sistema utiliza una lista de documentos en memoria, una función de recuperación por palabras clave y un LLM para generar respuestas.

In [ ]:
# Base de documentos
documents = [
    "La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana.",
    "Los modelos de lenguaje grande (LLM) son sistemas de IA entrenados en enormes cantidades de texto para generar y comprender lenguaje natural.",
    "RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.",
    "LangChain es un framework que facilita el desarrollo de aplicaciones con modelos de lenguaje, proporcionando herramientas para cadenas y agentes.",
    "El prompt engineering es la práctica de diseñar instrucciones efectivas para obtener los mejores resultados de los modelos de IA."
]

# Cliente de OpenAI
def initialize_client():
    client = OpenAI(
        base_url=os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1"),
        api_key=os.getenv("OPENAI_API_KEY")
    )
    return client

openai_client = initialize_client()

print(f"📚 Base de datos con {len(documents)} documentos cargada.")
print("✅ Cliente OpenAI inicializado correctamente.")

In [ ]:
from langsmith.run_helpers import traceable

# Envolvemos las funciones con el decorador @traceable para que LangSmith las capture

@traceable(name="Recuperacion de Documentos")
def simple_retrieval(query: str, documents: List[str]) -> List[str]:
    """Recupera documentos relevantes basándose en palabras clave."""
    relevant_docs = []
    query_lower = query.lower()
    for doc in documents:
        if any(word in doc.lower() for word in query_lower.split()):
            relevant_docs.append(doc)
    return relevant_docs[:3]

@traceable(name="Generacion de Respuesta")
def generate_response(client, query: str, context: str) -> str:
    """Genera una respuesta basada en el contexto proporcionado."""
    prompt = f"""Contexto:
{context}

Pregunta: {query}

Responde basándote únicamente en el contexto proporcionado. Si la información no está disponible en el contexto, indica que no puedes responder basándote en la información proporcionada."""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error al generar respuesta: {e}")
        return "Error al generar la respuesta."

@traceable(name="Pipeline RAG Completo")
def rag_pipeline(query: str) -> Dict[str, Any]:
    """Pipeline completo de RAG que incluye recuperación y generación."""
    context_docs = simple_retrieval(query, documents)
    context = "\n\n".join(context_docs)
    answer = generate_response(openai_client, query, context)
    return {
        "answer": answer, 
        "context": context_docs,
        "query": query
    }

print("✅ Funciones del pipeline RAG definidas y trazables")

### Prueba de Trazabilidad

Ahora, si ejecutamos nuestro pipeline, LangSmith registrará la ejecución completa, incluyendo los pasos intermedios que decoramos. Puedes ir a tu proyecto en LangSmith para ver la traza.

In [ ]:
# Prueba del sistema
resultado = rag_pipeline("¿Qué es RAG?")
print("Respuesta:")
print(resultado["answer"])
print("\nDocumentos recuperados:")
for i, doc in enumerate(resultado["context"], 1):
    print(f"{i}. {doc[:100]}...")

## 3. Creación de un Dataset de Evaluación

Para evaluar nuestro sistema, necesitamos un "ground truth" (verdad fundamental), es decir, un conjunto de preguntas y las respuestas que consideramos correctas. En LangSmith, esto se gestiona a través de Datasets.

### Tipos de Datasets de Evaluación

1. **Dataset de Exactitud**: Preguntas con respuestas específicas y correctas
2. **Dataset de Relevancia**: Evalúa si la respuesta es útil, aunque no sea exacta
3. **Dataset de Fidelidad**: Verifica si la respuesta se basa en el contexto proporcionado
4. **Dataset de Completitud**: Evalúa si la respuesta aborda todos los aspectos de la pregunta

### Mejores Prácticas para Crear Datasets:
- **Diversidad**: Incluir diferentes tipos de preguntas
- **Dificultad variada**: Preguntas fáciles, medianas y difíciles
- **Casos edge**: Preguntas que podrían confundir al sistema
- **Actualización continua**: Añadir nuevos ejemplos basados en casos reales

In [ ]:
# Dataset de evaluación más completo y estructurado
def create_comprehensive_evaluation_dataset():
    """Crea un dataset de evaluación más completo con diferentes categorías."""
    
    evaluation_examples = [
        # Categoría: Definiciones básicas
        {
            "inputs": {"query": "¿Qué es la inteligencia artificial?"},
            "outputs": {"answer": "La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana."},
            "metadata": {"category": "definicion", "difficulty": "facil", "expected_docs": 1}
        },
        {
            "inputs": {"query": "¿Qué son los LLM?"},
            "outputs": {"answer": "Los modelos de lenguaje grande (LLM) son sistemas de IA entrenados en enormes cantidades de texto para generar y comprender lenguaje natural."},
            "metadata": {"category": "definicion", "difficulty": "facil", "expected_docs": 1}
        },
        
        # Categoría: Aplicaciones y uso
        {
            "inputs": {"query": "¿Para qué sirve LangChain?"},
            "outputs": {"answer": "LangChain es un framework que facilita el desarrollo de aplicaciones con modelos de lenguaje, proporcionando herramientas para cadenas y agentes."},
            "metadata": {"category": "aplicacion", "difficulty": "medio", "expected_docs": 1}
        },
        {
            "inputs": {"query": "Explica qué es RAG y para qué se usa"},
            "outputs": {"answer": "RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas."},
            "metadata": {"category": "aplicacion", "difficulty": "medio", "expected_docs": 1}
        },
        
        # Categoría: Técnicas y métodos
        {
            "inputs": {"query": "¿Qué es prompt engineering?"},
            "outputs": {"answer": "El prompt engineering es la práctica de diseñar instrucciones efectivas para obtener los mejores resultados de los modelos de IA."},
            "metadata": {"category": "tecnica", "difficulty": "medio", "expected_docs": 1}
        },
        
        # Categoría: Preguntas combinadas (más difíciles)
        {
            "inputs": {"query": "¿Cómo se relacionan RAG y LangChain?"},
            "outputs": {"answer": "LangChain facilita la implementación de sistemas RAG proporcionando herramientas para combinar la búsqueda de información con la generación de respuestas usando modelos de lenguaje."},
            "metadata": {"category": "relacion", "difficulty": "dificil", "expected_docs": 2}
        },
        
        # Categoría: Casos edge - preguntas que no tienen respuesta en los docs
        {
            "inputs": {"query": "¿Cuál es el precio de OpenAI GPT-4?"},
            "outputs": {"answer": "No se puede responder esta pregunta basándose en la información proporcionada en el contexto."},
            "metadata": {"category": "sin_respuesta", "difficulty": "dificil", "expected_docs": 0}
        },
        
        # Categoría: Preguntas ambiguas
        {
            "inputs": {"query": "¿Qué es IA?"},
            "outputs": {"answer": "La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana."},
            "metadata": {"category": "ambigua", "difficulty": "facil", "expected_docs": 1}
        }
    ]
    
    return evaluation_examples

# Crear el dataset completo
comprehensive_examples = create_comprehensive_evaluation_dataset()

print(f"📊 Dataset completo con {len(comprehensive_examples)} ejemplos creado")
print("\n📈 Distribución por categoría:")
categories = {}
difficulties = {}

for example in comprehensive_examples:
    cat = example["metadata"]["category"]
    diff = example["metadata"]["difficulty"]
    categories[cat] = categories.get(cat, 0) + 1
    difficulties[diff] = difficulties.get(diff, 0) + 1

for cat, count in categories.items():
    print(f"  • {cat}: {count} ejemplos")
    
print(f"\n🎯 Distribución por dificultad:")
for diff, count in difficulties.items():
    print(f"  • {diff}: {count} ejemplos")

In [ ]:
def validate_dataset(examples):
    """Valida la calidad del dataset de evaluación."""
    
    print("🔍 Validando dataset de evaluación...")
    print("=" * 50)
    
    issues = []
    
    # Validar estructura
    for i, example in enumerate(examples):
        # Verificar estructura requerida
        if "inputs" not in example or "outputs" not in example:
            issues.append(f"Ejemplo {i+1}: Estructura inválida (falta inputs/outputs)")
        
        # Verificar que query no esté vacía
        if "query" not in example.get("inputs", {}) or not example["inputs"]["query"].strip():
            issues.append(f"Ejemplo {i+1}: Query vacía o faltante")
        
        # Verificar que answer no esté vacía
        if "answer" not in example.get("outputs", {}) or not example["outputs"]["answer"].strip():
            issues.append(f"Ejemplo {i+1}: Answer vacía o faltante")
        
        # Verificar longitud razonable
        query = example.get("inputs", {}).get("query", "")
        answer = example.get("outputs", {}).get("answer", "")
        
        if len(query) < 10:
            issues.append(f"Ejemplo {i+1}: Query muy corta (menos de 10 caracteres)")
        
        if len(answer) < 20:
            issues.append(f"Ejemplo {i+1}: Answer muy corta (menos de 20 caracteres)")
        
        if len(query) > 200:
            issues.append(f"Ejemplo {i+1}: Query muy larga (más de 200 caracteres)")
    
    # Verificar diversidad
    queries = [ex["inputs"]["query"].lower() for ex in examples]
    unique_queries = set(queries)
    
    if len(unique_queries) < len(queries):
        issues.append("Dataset contiene queries duplicadas")
    
    # Verificar categorías si existen metadatos
    if examples and "metadata" in examples[0]:
        categories = [ex["metadata"].get("category", "unknown") for ex in examples]
        unique_categories = set(categories)
        
        if len(unique_categories) == 1:
            issues.append("Dataset tiene poca diversidad de categorías")
    
    # Mostrar resultados
    if issues:
        print("❌ Problemas encontrados:")
        for issue in issues:
            print(f"  • {issue}")
    else:
        print("✅ Dataset válido - No se encontraron problemas")
    
    print(f"\n📊 Estadísticas del dataset:")
    print(f"  • Total de ejemplos: {len(examples)}")
    print(f"  • Queries únicas: {len(unique_queries)}")
    
    if examples and "metadata" in examples[0]:
        categories = {}
        difficulties = {}
        for ex in examples:
            cat = ex["metadata"].get("category", "unknown")
            diff = ex["metadata"].get("difficulty", "unknown")
            categories[cat] = categories.get(cat, 0) + 1
            difficulties[diff] = difficulties.get(diff, 0) + 1
        
        print(f"  • Categorías: {list(categories.keys())}")
        print(f"  • Dificultades: {list(difficulties.keys())}")
    
    return len(issues) == 0

# Validar nuestro dataset
is_valid = validate_dataset(comprehensive_examples)

In [ ]:
def create_langsmith_dataset_advanced():
    """Crear dataset en LangSmith con metadatos y gestión avanzada."""
    
    if not client:
        print("⚠️ LangSmith no configurado")
        return None
    
    dataset_name = "RAG Evaluation - Comprehensive Dataset"
    description = """Dataset completo para evaluación de sistemas RAG incluyendo:
    - Definiciones básicas
    - Aplicaciones y casos de uso
    - Técnicas y métodos
    - Preguntas combinadas
    - Casos edge y preguntas sin respuesta
    - Diferentes niveles de dificultad"""
    
    try:
        # Eliminar dataset existente si existe
        try:
            existing_dataset = client.read_dataset(dataset_name=dataset_name)
            client.delete_dataset(dataset_id=str(existing_dataset.id))
            print(f"🗑️ Dataset existente '{dataset_name}' eliminado.")
        except:
            pass
        
        # Crear nuevo dataset
        dataset = client.create_dataset(
            dataset_name=dataset_name,
            description=description,
        )
        
        print(f"✅ Dataset '{dataset_name}' creado con ID: {dataset.id}")
        
        # Añadir ejemplos con metadatos
        for i, example in enumerate(comprehensive_examples):
            try:
                client.create_example(
                    inputs=example["inputs"],
                    outputs=example["outputs"],
                    metadata=example["metadata"],  # Incluir metadatos
                    dataset_id=dataset.id,
                )
                print(f"  📝 Ejemplo {i+1} añadido: {example['metadata']['category']}")
            except Exception as e:
                print(f"  ❌ Error añadiendo ejemplo {i+1}: {e}")
        
        print(f"✅ {len(comprehensive_examples)} ejemplos añadidos exitosamente")
        
        return dataset
        
    except Exception as e:
        print(f"❌ Error creando dataset: {e}")
        return None

# Crear dataset avanzado
advanced_dataset = create_langsmith_dataset_advanced()

## 4. Evaluación Manual (Sin LangSmith)

Si no tienes LangSmith configurado, puedes realizar una evaluación básica manual:

In [ ]:
def simple_evaluation():
    """Evaluación simple sin LangSmith"""
    results = []
    
    print("🔍 Ejecutando evaluación manual...\n")
    
    for i, example in enumerate(comprehensive_examples, 1):
        query = example["inputs"]["query"]
        expected = example["outputs"]["answer"]
        category = example["metadata"]["category"]
        difficulty = example["metadata"]["difficulty"]
        
        print(f"Ejemplo {i}/{len(comprehensive_examples)} - {category} ({difficulty})")
        print(f"Pregunta: {query}")
        
        # Ejecutar el pipeline
        result = rag_pipeline(query)
        generated = result["answer"]
        
        print(f"Respuesta esperada: {expected}")
        print(f"Respuesta generada: {generated}")
        
        # Evaluación básica de similitud (palabras en común)
        expected_words = set(expected.lower().split())
        generated_words = set(generated.lower().split())
        
        if len(expected_words) > 0:
            similarity = len(expected_words.intersection(generated_words)) / len(expected_words.union(generated_words))
        else:
            similarity = 0
        
        # Evaluación de recuperación
        context_retrieved = len(result["context"]) > 0
        expected_docs = example["metadata"]["expected_docs"]
        retrieval_success = len(result["context"]) >= expected_docs if expected_docs > 0 else len(result["context"]) == 0
        
        results.append({
            "query": query,
            "expected": expected,
            "generated": generated,
            "similarity": similarity,
            "context_docs": len(result["context"]),
            "category": category,
            "difficulty": difficulty,
            "retrieval_success": retrieval_success
        })
        
        print(f"Similitud: {similarity:.2f}")
        print(f"Documentos recuperados: {len(result['context'])}")
        print(f"Recuperación exitosa: {'✅' if retrieval_success else '❌'}")
        print("-" * 80 + "\n")
    
    # Resumen por categoría
    print("📊 Resumen por Categoría:")
    print("=" * 50)
    
    category_stats = {}
    for r in results:
        cat = r["category"]
        if cat not in category_stats:
            category_stats[cat] = {"total": 0, "similarity_sum": 0, "retrieval_success": 0}
        
        category_stats[cat]["total"] += 1
        category_stats[cat]["similarity_sum"] += r["similarity"]
        if r["retrieval_success"]:
            category_stats[cat]["retrieval_success"] += 1
    
    for cat, stats in category_stats.items():
        avg_similarity = stats["similarity_sum"] / stats["total"]
        retrieval_rate = stats["retrieval_success"] / stats["total"]
        print(f"{cat}: Similitud={avg_similarity:.2f}, Recuperación={retrieval_rate:.2f}")
    
    # Resumen general
    avg_similarity = sum(r["similarity"] for r in results) / len(results)
    retrieval_rate = sum(r["retrieval_success"] for r in results) / len(results)
    
    print(f"\n📈 Resumen General:")
    print(f"Similitud promedio: {avg_similarity:.2f}")
    print(f"Tasa de recuperación exitosa: {retrieval_rate:.2f}")
    print(f"Total de ejemplos: {len(results)}")
    
    return results

# Ejecutar evaluación manual
manual_results = simple_evaluation()

## 5. Evaluación con LangSmith (Si está configurado)

Si tienes LangSmith configurado, puedes ejecutar una evaluación más sofisticada:

In [ ]:
# Solo ejecutar si LangSmith está configurado
if client and advanced_dataset:
    try:
        from langsmith.evaluation import evaluate
        from langchain_openai import ChatOpenAI
        
        print("🚀 Configurando evaluación con LangSmith...")
        
        # Modelo de lenguaje para la evaluación
        llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0.1,
            base_url=os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1"),
            api_key=os.getenv("OPENAI_API_KEY")
        )
        
        # Función target simplificada
        def target_function(inputs: Dict[str, Any]) -> str:
            """Función target que retorna solo la respuesta como string."""
            result = rag_pipeline(inputs["query"])
            return result["answer"]  # Retornar solo la respuesta como string
        
        print("🔄 Ejecutando evaluación sin evaluadores automáticos...")
        
        # ENFOQUE SIMPLIFICADO: Solo ejecutar y capturar las trazas
        experiment_results = evaluate(
            target_function,
            data="RAG Evaluation - Comprehensive Dataset",
            evaluators=[],  # Sin evaluadores automáticos por ahora
            experiment_prefix="RAG Basico - Trace Only",
            metadata={
                "version": "2.2.0", 
                "model": "gpt-4o", 
                "dataset": "comprehensive", 
                "approach": "trace_only"
            }
        )
        
        print("✅ Evaluación de trazabilidad completada exitosamente.")
        print("🔗 Revisa las trazas en la plataforma LangSmith.")
        print("📝 Las respuestas están capturadas - puedes evaluar manualmente en la interfaz")
        
        # Mostrar información del experimento
        if hasattr(experiment_results, 'experiment_name'):
            print(f"📊 Nombre del experimento: {experiment_results.experiment_name}")
            
    except Exception as e:
        print(f"❌ Error en evaluación con LangSmith: {e}")
        print("🔄 Continuando con evaluación manual local...")
        
else:
    print("⚠️ LangSmith no configurado, usando evaluación manual únicamente.")

In [ ]:
def comprehensive_manual_evaluation():
    """Evaluación manual completa con múltiples métricas."""
    print("🔍 Ejecutando evaluación manual completa...")
    print("=" * 60)
    
    results = []
    total_time = 0
    
    for i, example in enumerate(comprehensive_examples, 1):
        query = example["inputs"]["query"]
        expected = example["outputs"]["answer"]
        category = example["metadata"]["category"]
        difficulty = example["metadata"]["difficulty"]
        
        print(f"\n📝 Ejemplo {i}/{len(comprehensive_examples)}")
        print(f"Categoría: {category} | Dificultad: {difficulty}")
        print(f"Pregunta: {query}")
        
        # Medir tiempo de respuesta
        import time
        start_time = time.time()
        
        # Ejecutar pipeline
        result = rag_pipeline(query)
        
        end_time = time.time()
        response_time = end_time - start_time
        total_time += response_time
        
        generated = result["answer"]
        context_docs = result["context"]
        
        print(f"Respuesta generada: {generated}")
        print(f"Respuesta esperada: {expected}")
        print(f"Documentos recuperados: {len(context_docs)}")
        print(f"Tiempo de respuesta: {response_time:.2f}s")
        
        # Múltiples métricas de evaluación
        
        # 1. Similitud de palabras (Jaccard)
        expected_words = set(expected.lower().split())
        generated_words = set(generated.lower().split())
        
        if len(expected_words) > 0:
            jaccard_similarity = len(expected_words.intersection(generated_words)) / len(expected_words.union(generated_words))
        else:
            jaccard_similarity = 0
        
        # 2. Contención (¿La respuesta contiene conceptos clave?)
        key_concepts = ["inteligencia artificial", "llm", "rag", "langchain", "prompt engineering"]
        concepts_in_expected = [concept for concept in key_concepts if concept in expected.lower()]
        concepts_in_generated = [concept for concept in key_concepts if concept in generated.lower()]
        
        concept_coverage = len(concepts_in_generated) / max(1, len(concepts_in_expected)) if concepts_in_expected else 1
        
        # 3. Evaluación de recuperación
        expected_docs = example["metadata"]["expected_docs"]
        retrieval_success = len(context_docs) >= expected_docs if expected_docs > 0 else len(context_docs) == 0
        
        # 4. Longitud de respuesta (relativa a la esperada)
        length_ratio = len(generated) / max(1, len(expected))
        length_score = 1.0 if 0.5 <= length_ratio <= 2.0 else max(0, 1 - abs(length_ratio - 1))
        
        # 5. Detección de "no puedo responder"
        no_answer_phrases = ["no puedo responder", "no se puede responder", "no está disponible", "no tengo información"]
        contains_no_answer = any(phrase in generated.lower() for phrase in no_answer_phrases)
        should_have_no_answer = expected_docs == 0
        no_answer_correct = (contains_no_answer and should_have_no_answer) or (not contains_no_answer and not should_have_no_answer)
        
        # Puntuación compuesta
        composite_score = (
            jaccard_similarity * 0.3 +
            concept_coverage * 0.2 +
            (1.0 if retrieval_success else 0.0) * 0.2 +
            length_score * 0.15 +
            (1.0 if no_answer_correct else 0.0) * 0.15
        )
        
        result_data = {
            "query": query,
            "expected": expected,
            "generated": generated,
            "category": category,
            "difficulty": difficulty,
            "context_docs": len(context_docs),
            "response_time": response_time,
            "jaccard_similarity": jaccard_similarity,
            "concept_coverage": concept_coverage,
            "retrieval_success": retrieval_success,
            "length_score": length_score,
            "no_answer_correct": no_answer_correct,
            "composite_score": composite_score
        }
        
        results.append(result_data)
        
        print(f"Similitud Jaccard: {jaccard_similarity:.3f}")
        print(f"Cobertura conceptos: {concept_coverage:.3f}")
        print(f"Recuperación exitosa: {'✅' if retrieval_success else '❌'}")
        print(f"Puntuación compuesta: {composite_score:.3f}")
        print("-" * 50)
    
    # Análisis de resultados
    print(f"\n📊 RESUMEN DE EVALUACIÓN COMPLETA")
    print("=" * 60)
    
    avg_jaccard = sum(r["jaccard_similarity"] for r in results) / len(results)
    avg_concept = sum(r["concept_coverage"] for r in results) / len(results)
    avg_composite = sum(r["composite_score"] for r in results) / len(results)
    avg_time = total_time / len(results)
    retrieval_rate = sum(r["retrieval_success"] for r in results) / len(results)
    
    print(f"Similitud Jaccard promedio: {avg_jaccard:.3f}")
    print(f"Cobertura de conceptos promedio: {avg_concept:.3f}")
    print(f"Tasa de recuperación exitosa: {retrieval_rate:.3f}")
    print(f"Tiempo promedio de respuesta: {avg_time:.2f}s")
    print(f"Puntuación compuesta promedio: {avg_composite:.3f}")
    
    # Análisis por categoría
    print(f"\n📈 Análisis por categoría:")
    category_stats = {}
    for r in results:
        cat = r["category"]
        if cat not in category_stats:
            category_stats[cat] = []
        category_stats[cat].append(r["composite_score"])
    
    for cat, scores in category_stats.items():
        avg_score = sum(scores) / len(scores)
        print(f"  {cat}: {avg_score:.3f} ({len(scores)} ejemplos)")
    
    # Análisis por dificultad
    print(f"\n🎯 Análisis por dificultad:")
    difficulty_stats = {}
    for r in results:
        diff = r["difficulty"]
        if diff not in difficulty_stats:
            difficulty_stats[diff] = []
        difficulty_stats[diff].append(r["composite_score"])
    
    for diff, scores in difficulty_stats.items():
        avg_score = sum(scores) / len(scores)
        print(f"  {diff}: {avg_score:.3f} ({len(scores)} ejemplos)")
    
    return results

# Ejecutar evaluación manual completa
comprehensive_results = comprehensive_manual_evaluation()

## 6. Análisis de Resultados y Mejoras

Basándose en los resultados de la evaluación, aquí tienes algunas áreas comunes de mejora para sistemas RAG:

In [ ]:
def analyze_results_detailed(results):
    """Analiza los resultados y sugiere mejoras específicas."""
    print("📈 Análisis Detallado de Resultados:")
    print("=" * 50)
    
    if not results:
        print("⚠️ No hay resultados para analizar")
        return
    
    # Estadísticas generales
    avg_composite = sum(r["composite_score"] for r in results) / len(results)
    avg_jaccard = sum(r["jaccard_similarity"] for r in results) / len(results)
    retrieval_rate = sum(r["retrieval_success"] for r in results) / len(results)
    avg_time = sum(r["response_time"] for r in results) / len(results)
    
    print(f"Puntuación compuesta promedio: {avg_composite:.3f}")
    print(f"Similitud Jaccard promedio: {avg_jaccard:.3f}")
    print(f"Tasa de recuperación exitosa: {retrieval_rate:.3f}")
    print(f"Tiempo promedio de respuesta: {avg_time:.2f}s")
    
    # Identificar problemas específicos
    problems = []
    
    if avg_composite < 0.6:
        problems.append("Puntuación general baja")
    if avg_jaccard < 0.3:
        problems.append("Baja similitud en las respuestas")
    if retrieval_rate < 0.7:
        problems.append("Problemas en la recuperación de documentos")
    if avg_time > 5.0:
        problems.append("Tiempo de respuesta lento")
    
    # Encontrar casos problemáticos
    low_performance = [r for r in results if r["composite_score"] < 0.4]
    retrieval_failures = [r for r in results if not r["retrieval_success"]]
    
    print(f"\n🚨 Problemas identificados:")
    if problems:
        for problem in problems:
            print(f"  • {problem}")
    else:
        print("  ✅ No se identificaron problemas mayores")
    
    if low_performance:
        print(f"\n📉 Casos de bajo rendimiento ({len(low_performance)}):")
        for case in low_performance[:3]:  # Mostrar solo los primeros 3
            print(f"  • '{case['query'][:50]}...' (Score: {case['composite_score']:.3f})")
    
    if retrieval_failures:
        print(f"\n🔍 Fallos de recuperación ({len(retrieval_failures)}):")
        for case in retrieval_failures[:3]:
            print(f"  • '{case['query'][:50]}...' (Docs: {case['context_docs']})")
    
    # Recomendaciones específicas
    print(f"\n💡 Recomendaciones específicas:")
    
    if avg_jaccard < 0.3:
        print("  🔧 Mejorar la generación de respuestas:")
        print("    - Refinar el prompt de generación")
        print("    - Usar ejemplos few-shot")
        print("    - Ajustar la temperatura del modelo")
    
    if retrieval_rate < 0.7:
        print("  🔧 Mejorar la recuperación:")
        print("    - Implementar búsqueda semántica con embeddings")
        print("    - Ajustar umbrales de similitud")
        print("    - Expandir la base de documentos")
    
    if avg_time > 3.0:
        print("  🔧 Optimizar rendimiento:")
        print("    - Usar modelos más rápidos para recuperación")
        print("    - Implementar cache de respuestas")
        print("    - Paralelizar operaciones cuando sea posible")
    
    print(f"\n🔧 Estrategias generales de mejora:")
    print("1. Implementar embeddings semánticos (OpenAI, Sentence Transformers)")
    print("2. Usar re-ranking de documentos recuperados")
    print("3. Implementar query expansion y refinement")
    print("4. Añadir evaluación de confianza en las respuestas")
    print("5. Crear un sistema de feedback para mejorar continuamente")

# Analizar resultados detallados
analyze_results_detailed(comprehensive_results)

## 7. Ejemplo de Mejora: Sistema RAG con Embeddings

Como ejemplo de mejora, aquí tienes una versión mejorada que usa embeddings semánticos:

In [ ]:
# Versión mejorada con TF-IDF (como alternativa a embeddings reales)
class ImprovedRAGSystem:
    def __init__(self, documents):
        self.documents = documents
        self.vectorizer = TfidfVectorizer(
            stop_words='english',
            ngram_range=(1, 2),  # Usar unigramas y bigramas
            max_features=1000,   # Limitar características
            lowercase=True
        )
        self.doc_vectors = self.vectorizer.fit_transform(documents)
        print(f"✅ Sistema mejorado inicializado con {len(documents)} documentos")
    
    @traceable(name="Recuperacion Semantica Mejorada")
    def semantic_retrieval(self, query, top_k=3, threshold=0.1):
        """Recuperación basada en similitud semántica usando TF-IDF."""
        query_vector = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vector, self.doc_vectors)[0]
        
        # Obtener los índices de los documentos más similares
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        relevant_docs = []
        similarity_scores = []
        
        for i in top_indices:
            if similarities[i] > threshold:
                relevant_docs.append(self.documents[i])
                similarity_scores.append(similarities[i])
        
        return relevant_docs, similarity_scores
    
    @traceable(name="Generacion de Respuesta Mejorada")
    def generate_enhanced_response(self, query, context_docs, similarity_scores):
        """Genera respuesta con contexto enriquecido."""
        if not context_docs:
            return "No se encontró información relevante para responder la pregunta."
        
        # Crear contexto enriquecido con puntuaciones de similitud
        enriched_context = "Información relevante encontrada:\n\n"
        for i, (doc, score) in enumerate(zip(context_docs, similarity_scores), 1):
            enriched_context += f"Documento {i} (relevancia: {score:.3f}):\n{doc}\n\n"
        
        prompt = f"""Basándote en la información proporcionada, responde la siguiente pregunta de manera precisa y completa.

{enriched_context}

Pregunta: {query}

Instrucciones:
- Usa únicamente la información proporcionada en los documentos
- Si la información no está disponible, indica claramente que no puedes responder
- Sé preciso y directo en tu respuesta
- Si hay múltiples documentos relevantes, sintetiza la información

Respuesta:"""
        
        try:
            response = openai_client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                max_tokens=300
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error al generar respuesta: {e}")
            return "Error al generar la respuesta."
    
    @traceable(name="Pipeline RAG Mejorado Completo")
    def improved_rag_pipeline(self, query):
        """Pipeline RAG mejorado con recuperación semántica y generación enriquecida."""
        context_docs, similarity_scores = self.semantic_retrieval(query)
        answer = self.generate_enhanced_response(query, context_docs, similarity_scores)
        
        return {
            "answer": answer,
            "context": context_docs,
            "similarity_scores": similarity_scores,
            "query": query,
            "num_docs_retrieved": len(context_docs)
        }

# Crear sistema mejorado
improved_system = ImprovedRAGSystem(documents)

print("✅ Sistema RAG mejorado creado con TF-IDF semántico")

In [ ]:
def compare_systems_comprehensive():
    """Compara el sistema original con el mejorado de forma detallada."""
    print("🔍 Comparación Comprensiva de Sistemas RAG")
    print("=" * 60)
    
    test_queries = [
        "¿Qué es la inteligencia artificial?",
        "¿Cómo funciona RAG?", 
        "Explica los modelos de lenguaje",
        "¿Qué herramientas usa LangChain?",
        "¿Qué es el deep learning?"  # Esta no debería tener respuesta clara
    ]
    
    comparison_results = []
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n🔍 Prueba {i}: '{query}'")
        print("-" * 50)
        
        # Medir tiempo sistema original
        import time
        start_time = time.time()
        original_result = rag_pipeline(query)
        original_time = time.time() - start_time
        
        # Medir tiempo sistema mejorado
        start_time = time.time()
        improved_result = improved_system.improved_rag_pipeline(query)
        improved_time = time.time() - start_time
        
        print("Sistema Original:")
        print(f"  Respuesta: {original_result['answer'][:200]}...")
        print(f"  Documentos recuperados: {len(original_result['context'])}")
        print(f"  Tiempo: {original_time:.3f}s")
        
        print("\nSistema Mejorado:")
        print(f"  Respuesta: {improved_result['answer'][:200]}...")
        print(f"  Documentos recuperados: {len(improved_result['context'])}")
        if improved_result.get('similarity_scores'):
            avg_similarity = sum(improved_result['similarity_scores']) / len(improved_result['similarity_scores'])
            print(f"  Similitud promedio: {avg_similarity:.3f}")
        print(f"  Tiempo: {improved_time:.3f}s")
        
        # Calcular métricas de comparación
        original_length = len(original_result['answer'])
        improved_length = len(improved_result['answer'])
        
        comparison_data = {
            "query": query,
            "original_docs": len(original_result['context']),
            "improved_docs": len(improved_result['context']),
            "original_time": original_time,
            "improved_time": improved_time,
            "original_length": original_length,
            "improved_length": improved_length,
            "time_improvement": ((original_time - improved_time) / original_time * 100) if original_time > 0 else 0
        }
        
        comparison_results.append(comparison_data)
        
        print(f"Mejora en tiempo: {comparison_data['time_improvement']:+.1f}%")
    
    # Resumen de comparación
    print(f"\n📊 RESUMEN DE COMPARACIÓN")
    print("=" * 40)
    
    avg_original_time = sum(r["original_time"] for r in comparison_results) / len(comparison_results)
    avg_improved_time = sum(r["improved_time"] for r in comparison_results) / len(comparison_results)
    avg_time_improvement = sum(r["time_improvement"] for r in comparison_results) / len(comparison_results)
    
    print(f"Tiempo promedio original: {avg_original_time:.3f}s")
    print(f"Tiempo promedio mejorado: {avg_improved_time:.3f}s")
    print(f"Mejora promedio en tiempo: {avg_time_improvement:+.1f}%")
    
    return comparison_results

# Ejecutar comparación
comparison_data = compare_systems_comprehensive()

In [ ]:
def evaluate_improved_system_complete():
    """Evalúa el sistema mejorado con el mismo dataset usando las mismas métricas."""
    print("🚀 Evaluando Sistema RAG Mejorado...")
    print("=" * 50)
    
    improved_results = []
    
    for i, example in enumerate(comprehensive_examples, 1):
        query = example["inputs"]["query"]
        expected = example["outputs"]["answer"]
        category = example["metadata"]["category"]
        difficulty = example["metadata"]["difficulty"]
        
        print(f"Evaluando ejemplo {i}/{len(comprehensive_examples)}: {category}")
        
        # Ejecutar pipeline mejorado
        import time
        start_time = time.time()
        result = improved_system.improved_rag_pipeline(query)
        response_time = time.time() - start_time
        
        generated = result["answer"]
        context_docs = result["context"]
        similarity_scores = result.get("similarity_scores", [])
        
        # Aplicar las mismas métricas que la evaluación manual
        expected_words = set(expected.lower().split())
        generated_words = set(generated.lower().split())
        
        if len(expected_words) > 0:
            jaccard_similarity = len(expected_words.intersection(generated_words)) / len(expected_words.union(generated_words))
        else:
            jaccard_similarity = 0
        
        # Cobertura de conceptos
        key_concepts = ["inteligencia artificial", "llm", "rag", "langchain", "prompt engineering"]
        concepts_in_expected = [concept for concept in key_concepts if concept in expected.lower()]
        concepts_in_generated = [concept for concept in key_concepts if concept in generated.lower()]
        concept_coverage = len(concepts_in_generated) / max(1, len(concepts_in_expected)) if concepts_in_expected else 1
        
        # Evaluación de recuperación
        expected_docs = example["metadata"]["expected_docs"]
        retrieval_success = len(context_docs) >= expected_docs if expected_docs > 0 else len(context_docs) == 0
        
        # Longitud de respuesta
        length_ratio = len(generated) / max(1, len(expected))
        length_score = 1.0 if 0.5 <= length_ratio <= 2.0 else max(0, 1 - abs(length_ratio - 1))
        
        # Detección de "no puedo responder"
        no_answer_phrases = ["no puedo responder", "no se puede responder", "no está disponible", "no tengo información"]
        contains_no_answer = any(phrase in generated.lower() for phrase in no_answer_phrases)
        should_have_no_answer = expected_docs == 0
        no_answer_correct = (contains_no_answer and should_have_no_answer) or (not contains_no_answer and not should_have_no_answer)
        
        # Puntuación compuesta (misma fórmula)
        composite_score = (
            jaccard_similarity * 0.3 +
            concept_coverage * 0.2 +
            (1.0 if retrieval_success else 0.0) * 0.2 +
            length_score * 0.15 +
            (1.0 if no_answer_correct else 0.0) * 0.15
        )
        
        # Métrica adicional: calidad de recuperación semántica
        avg_semantic_similarity = sum(similarity_scores) / len(similarity_scores) if similarity_scores else 0
        
        result_data = {
            "query": query,
            "expected": expected,
            "generated": generated,
            "category": category,
            "difficulty": difficulty,
            "context_docs": len(context_docs),
            "response_time": response_time,
            "jaccard_similarity": jaccard_similarity,
            "concept_coverage": concept_coverage,
            "retrieval_success": retrieval_success,
            "length_score": length_score,
            "no_answer_correct": no_answer_correct,
            "composite_score": composite_score,
            "avg_semantic_similarity": avg_semantic_similarity
        }
        
        improved_results.append(result_data)
    
    # Calcular estadísticas mejoradas
    avg_composite_improved = sum(r["composite_score"] for r in improved_results) / len(improved_results)
    avg_jaccard_improved = sum(r["jaccard_similarity"] for r in improved_results) / len(improved_results)
    retrieval_rate_improved = sum(r["retrieval_success"] for r in improved_results) / len(improved_results)
    avg_semantic_improved = sum(r["avg_semantic_similarity"] for r in improved_results) / len(improved_results)
    
    print(f"\n📊 Resultados del Sistema Mejorado:")
    print(f"Puntuación compuesta promedio: {avg_composite_improved:.3f}")
    print(f"Similitud Jaccard promedio: {avg_jaccard_improved:.3f}")
    print(f"Tasa de recuperación exitosa: {retrieval_rate_improved:.3f}")
    print(f"Similitud semántica promedio: {avg_semantic_improved:.3f}")
    
    # Comparar con resultados originales
    if 'comprehensive_results' in globals():
        original_avg = sum(r["composite_score"] for r in comprehensive_results) / len(comprehensive_results)
        original_jaccard = sum(r["jaccard_similarity"] for r in comprehensive_results) / len(comprehensive_results)
        original_retrieval = sum(r["retrieval_success"] for r in comprehensive_results) / len(comprehensive_results)
        
        print(f"\n📈 Comparación con Sistema Original:")
        print(f"Mejora en puntuación compuesta: {((avg_composite_improved - original_avg) / original_avg * 100):+.1f}%")
        print(f"Mejora en similitud Jaccard: {((avg_jaccard_improved - original_jaccard) / original_jaccard * 100):+.1f}%")
        print(f"Mejora en recuperación: {((retrieval_rate_improved - original_retrieval) / original_retrieval * 100):+.1f}%")
    
    return improved_results

# Evaluar sistema mejorado
improved_evaluation_results = evaluate_improved_system_complete()

### Añadir Ejemplos al Dataset

Cada ejemplo consta de:
- `inputs`: Un diccionario con las entradas de nuestro sistema (en este caso, la `query`).
- `outputs`: Un diccionario con la salida de referencia que esperamos (la `answer` correcta).

In [ ]:
client.create_example(
    inputs={"query": "¿Qué es la inteligencia artificial?"},
    outputs={"answer": "La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana."},
    dataset_id=dataset.id,
)

client.create_example(
    inputs={"query": "¿Para qué sirve LangChain?"},
    outputs={"answer": "LangChain es un framework que facilita el desarrollo de aplicaciones con modelos de lenguaje, proporcionando herramientas para cadenas y agentes."},
    dataset_id=dataset.id,
)

client.create_example(
    inputs={"query": "Explica qué es RAG"},
    outputs={"answer": "RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas."},
    dataset_id=dataset.id,
)

print(f"✅ 3 ejemplos añadidos al dataset '{dataset_name}'.")

## 8. Análisis de Resultados

Una vez completada la evaluación, puedes navegar a la pestaña **Experiments** en tu proyecto de LangSmith.

Allí encontrarás:
1. Un resumen del experimento con las puntuaciones medias de cada métrica.
2. Una tabla detallada con cada pregunta del dataset, la respuesta generada, la respuesta de referencia y las puntuaciones de la evaluación.
3. Para cada fila, puedes hacer clic para ver la traza completa y entender por qué el sistema respondió de esa manera (qué documentos recuperó, qué prompt se usó, etc.).

Este análisis te permite identificar puntos débiles. Por ejemplo:
- **Puntuaciones bajas de `correctness`**: Puede que la recuperación no esté funcionando bien o que el prompt de generación necesite ajustes.
- **Documentos irrelevantes en el contexto**: Indica que el método de `retrieval` debe ser mejorado (por ejemplo, pasando de búsqueda por palabras clave a búsqueda semántica con embeddings).

## Conclusión

La evaluación es un pilar fundamental en el desarrollo de sistemas de IA robustos. LangSmith nos ofrece un conjunto de herramientas poderosas para automatizar este proceso en sistemas RAG, permitiéndonos pasar de un desarrollo basado en la intuición a uno guiado por datos y métricas objetivas. Con este enfoque, podemos mejorar de forma iterativa la calidad y fiabilidad de nuestras aplicaciones.